In [18]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.preprocessing import MinMaxScaler

In [19]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', None)

In [20]:
df_enrolment = pd.read_csv('enrolment.csv')
df_demographic = pd.read_csv('demographic.csv')
df_biometric = pd.read_csv('biometric.csv')

In [21]:
for df in [df_enrolment, df_demographic, df_biometric]:
    df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
    df['month'] = df['date'].dt.month

In [22]:
def clean_district(name):
    if pd.isna(name):
        return name
    name = str(name).lower()
    name = name.replace('*', '')
    name = re.sub(r'\s+', ' ', name)
    return name.strip()

for df in [df_enrolment, df_demographic, df_biometric]:
    df['district'] = df['district'].apply(clean_district)

canonical_map = {
    'bokaro': 'bokaro',
    'chatra': 'chatra',
    'deoghar': 'deoghar',
    'dhanbad': 'dhanbad',
    'dumka': 'dumka',
    'east singhbhum': 'east singhbhum',
    'purbi singhbhum': 'east singhbhum',
    'garhwa': 'garhwa',
    'giridih': 'giridih',
    'godda': 'godda',
    'gumla': 'gumla',
    'hazaribagh': 'hazaribagh',
    'hazaribag': 'hazaribagh',
    'jamtara': 'jamtara',
    'khunti': 'khunti',
    'koderma': 'koderma',
    'kodarma': 'koderma',
    'latehar': 'latehar',
    'lohardaga': 'lohardaga',
    'pakur': 'pakur',
    'pakaur': 'pakur',
    'palamu': 'palamu',
    'ramgarh': 'ramgarh',
    'ranchi': 'ranchi',
    'sahibganj': 'sahibganj',
    'sahebganj': 'sahibganj',
    'seraikela kharsawan': 'seraikela kharsawan',
    'saraikela kharsawan': 'seraikela kharsawan',
    'saraikela-kharsawan': 'seraikela kharsawan',
    'simdega': 'simdega',
    'west singhbhum': 'west singhbhum',
    'east singhbum': 'east singhbhum',
    'palamau': 'palamu',
    'pashchimi singhbhum': 'west singhbhum',
    'pashchim singhbhum': 'west singhbhum'
}

for df in [df_enrolment, df_demographic, df_biometric]:
    df['district'] = df['district'].replace(canonical_map)

all_districts = sorted(set(df_enrolment['district']) | set(df_demographic['district']) | set(df_biometric['district']))
print("Total unique districts found:", len(all_districts))
for d in all_districts:
    print(d)

expected_count = 24
if len(all_districts) != expected_count:
    print(f"WARNING: Expected {expected_count} districts, found {len(all_districts)}. Please check the list above.")
else:
    print(f"✓ Jharkhand district count locked at {expected_count}")

Total unique districts found: 24
bokaro
chatra
deoghar
dhanbad
dumka
east singhbhum
garhwa
giridih
godda
gumla
hazaribagh
jamtara
khunti
koderma
latehar
lohardaga
pakur
palamu
ramgarh
ranchi
sahibganj
seraikela-kharsawan
simdega
west singhbhum
✓ Jharkhand district count locked at 24


In [23]:
enrol_agg = df_enrolment.groupby(["state", "district", "month"])[["age_0_5", "age_5_17", "age_18_greater"]].sum().reset_index()
demo_agg = df_demographic.groupby(["state", "district", "month"])[["demo_age_5_17", "demo_age_17_"]].sum().reset_index()
bio_agg = df_biometric.groupby(["state", "district", "month"])[["bio_age_5_17", "bio_age_17_"]].sum().reset_index()

combined_df = enrol_agg.merge(demo_agg, on = ["state", "district", "month"], how = "left").merge(bio_agg, on = ["state", "district", "month"], how = "left")
combined_df.fillna(0, inplace=True)

combined_df["E"] = combined_df["age_0_5"] + combined_df["age_5_17"] + combined_df["age_18_greater"]
combined_df["DU"] = combined_df["demo_age_5_17"] + combined_df["demo_age_17_"]
combined_df["BU"] = combined_df["bio_age_5_17"] + combined_df["bio_age_17_"]
combined_df["U"] = combined_df["DU"] + combined_df["BU"]
combined_df["T"] = combined_df["E"] + combined_df["U"]

In [24]:
district_monthly_counts = combined_df.groupby(["district", "month"]).agg(
    total_months = ("month", "count"), 
    active_months = ("T", lambda x : (x > 0).sum())
).reset_index()

district_monthly_counts["zero_months"] = district_monthly_counts["total_months"] - district_monthly_counts["active_months"]
district_monthly_counts["activity_ratio"] = district_monthly_counts["active_months"] / district_monthly_counts["total_months"]
district_monthly_counts["zero_month_ratio"] = district_monthly_counts["zero_months"] / district_monthly_counts["total_months"]

combined_df = combined_df.merge(district_monthly_counts[["district", "month", "activity_ratio", "zero_month_ratio"]], on = ["district", "month"], how = "left")

In [25]:
district_volume_metrics = combined_df.groupby(["state", "district"]).agg(
    avg_monthly_enrolment = ("E", "mean"),
    monthly_valatility = ("T", lambda x: x.std(ddof=0) / x.mean() if x.mean() > 0 else 0),
    peak_load_ratio = ("T", lambda x: x.max() / x.mean() if x.mean() > 0 else 0)
).reset_index()

combined_df = combined_df.merge(district_volume_metrics, on=["state", "district"], how="left")

In [26]:
district_update_burden = combined_df.groupby(["state", "district"]).agg(
    avg_monthly_enrollments = ("E", "sum"), 
    avg_monthly_demo_updates = ("DU", "sum"), 
    avg_monthly_bio_updates = ("BU", "sum")
).reset_index()

district_update_burden["U"] = district_update_burden["avg_monthly_demo_updates"] + district_update_burden["avg_monthly_bio_updates"]
district_update_burden["biometric_burden"] = district_update_burden["avg_monthly_bio_updates"] / (district_update_burden["avg_monthly_bio_updates"] + district_update_burden["avg_monthly_demo_updates"])
district_update_burden["update_dominant"] = np.where(district_update_burden["U"] > district_update_burden["avg_monthly_enrollments"], 1, 0)
district_update_burden["enrollment_update_balance"] = district_update_burden["avg_monthly_enrollments"] / (district_update_burden["avg_monthly_enrollments"] + district_update_burden["U"])

combined_df = combined_df.merge(district_update_burden[["state", "district", "biometric_burden", "update_dominant", "enrollment_update_balance"]], on=["state", "district"], how="left")

cols_to_drop = [c for c in combined_df.columns if c.endswith('_y')]
if cols_to_drop:
    combined_df.drop(cols_to_drop, axis=1, inplace=True)
    combined_df.rename(columns={c: c.replace('_x', '') for c in combined_df.columns if c.endswith('_x')}, inplace=True)

In [27]:
combined_df = combined_df.groupby(["state", "district"], as_index = False).first()

def normalize(x):
    maxx, minx = x.max(), x.min()
    if maxx == minx:
        return x * 0 + 0.5
    normalized = (x - minx) / (maxx - minx)
    return normalized

def inverse_normalize(x):
    inversed = 1 - normalize(x)
    return inversed

combined_df["access"] = (combined_df["activity_ratio"] + normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["responsiveness"] = normalize(combined_df["U"] / (combined_df["E"] + combined_df["U"]))
combined_df["inclusion"] = normalize((combined_df["age_0_5"] + combined_df["age_5_17"]) / combined_df["E"])
combined_df["stability"] = (inverse_normalize(combined_df["monthly_valatility"]) + inverse_normalize(combined_df["peak_load_ratio"])) / 2
combined_df["visibility"] = combined_df["activity_ratio"]

combined_df["DEI"] = (combined_df["access"] + combined_df["responsiveness"] + combined_df["inclusion"] + combined_df["stability"] + combined_df["visibility"]) / 5
combined_df["ASS"] = (inverse_normalize(combined_df["activity_ratio"]) + inverse_normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["UBS"] = (normalize(combined_df["biometric_burden"]) + normalize(combined_df["update_dominant"])) / 2
combined_df["SRS"] = (normalize(combined_df["monthly_valatility"]) + normalize(combined_df["zero_month_ratio"])) / 2

combined_df.drop(["access", "responsiveness", "inclusion", "stability", "visibility"], axis=1, inplace=True)

In [28]:
print(f"Total districts: {len(combined_df)}")
for col in ['DEI', 'ASS', 'UBS', 'SRS']:
    if combined_df[col].isnull().any():
        combined_df[col].fillna(0, inplace=True)

combined_df.to_csv("jharkhand_district_analysis.csv", index=False)
final_df = combined_df[["state", "district", "DEI", "ASS", "UBS", "SRS"]]
final_df.to_csv("jharkhand_district_final_scores.csv", index=False)
print("Done")

Total districts: 24
Done
